# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [ ]:
import pandas as pd
import numpy as np
import math
import json
from tqdm import tqdm
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [63]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [83]:
portfolio.rename(columns={'id': 'offer'}, inplace=True)

In [105]:
for channel in ['email', 'mobile', 'social', 'web']:
    portfolio[channel] = portfolio['channels'].apply(lambda row: 1 if (channel in row) else 0)
# df.columns = ['team_{}'.format(x+1) for x in df.columns]

In [107]:
portfolio

,reward,channels,difficulty,duration,offer_type,offer,email,mobile,social,web
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


In [64]:
profile

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [ ]:
profile = profile[profile['income'].notnull()]
real_persons = profile.id.unique()

In [80]:
profile.rename(columns={'id': 'person'}, inplace=True)
profile

,gender,age,person,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [ ]:
profile.hist()

In [ ]:
transcript

In [ ]:
transcript['event'].value_counts()

In [ ]:
persons_completed = transcript[transcript['event'] == 'offer completed']['person'].unique()
persons_viewed = transcript[transcript['event'] == 'offer viewed']['person'].unique()
persons_received = transcript[transcript['event'] == 'offer received']['person'].unique()

In [ ]:
transcript['person'].value_counts()

In [ ]:
transcript_train = transcript[transcript['person'].isin(persons_received) & transcript['person'].isin(persons_viewed)]
transcript_train = transcript_train[transcript_train['person'].isin(real_persons)]

In [ ]:
transcript_train['person'].value_counts()

In [ ]:
transcript_train

In [ ]:
transcript_offers = transcript_train.copy()
transcript_offers = transcript_offers[(transcript_offers['event'] == 'offer received') | (transcript_offers['event'] == 'offer viewed') | (transcript_offers['event'] == 'offer completed')]

In [ ]:
transcript_offers

In [ ]:
df_offers = pd.DataFrame(columns=['person', 'offer_received', 'offer_viewed', 'offer_completed', 'offer_id'])

In [71]:
offers_df = pd.DataFrame()
for person in tqdm(real_persons):
    df_aux = transcript_offers[transcript_offers['person'] == person]

    offers = dict()
    for line in range(len(df_aux)):
        if df_aux.iloc[line, df_aux.columns.get_loc('event')] == 'offer received':
            offers[df_aux.iloc[line, df_aux.columns.get_loc('value')].get('offer id')] = dict()
            offers[df_aux.iloc[line, df_aux.columns.get_loc('value')].get('offer id')]['person'] = person

            offers[df_aux.iloc[line, df_aux.columns.get_loc('value')].get('offer id')]['offer received'] = df_aux.iloc[line, df_aux.columns.get_loc('time')]
        elif df_aux.iloc[line, df_aux.columns.get_loc('event')] == 'offer viewed':
            offers[df_aux.iloc[line, df_aux.columns.get_loc('value')].get('offer id')]['offer viewed'] = df_aux.iloc[line, df_aux.columns.get_loc('time')]
        else:
            offers[df_aux.iloc[line, df_aux.columns.get_loc('value')]['offer_id']]['offer completed'] = df_aux.iloc[line, df_aux.columns.get_loc('time')]
            offers[df_aux.iloc[line, df_aux.columns.get_loc('value')]['offer_id']]['reward'] = df_aux.iloc[line, df_aux.columns.get_loc('value')]['reward']

    offers_df = offers_df.append(pd.DataFrame(offers).transpose())

100%|██████████| 14825/14825 [12:35<00:00, 19.63it/s]


In [73]:
offers_df['offer'] = offers_df.index
offers_df

,person,offer received,offer viewed,offer completed,reward,offer
9b98b8c7a33c4b65b9aebfe6a799e6d9,78afa995795e4d85b5d9ceeca43f5fef,0,6,132,5,9b98b8c7a33c4b65b9aebfe6a799e6d9
5a8bc65990b245e5a138643cd4eb9837,78afa995795e4d85b5d9ceeca43f5fef,168,216,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
ae264e3637204a6fb9bb56bc8210ddfd,78afa995795e4d85b5d9ceeca43f5fef,408,408,510,10,ae264e3637204a6fb9bb56bc8210ddfd
f19421c1d4aa40978ebb69ca19b0e20d,78afa995795e4d85b5d9ceeca43f5fef,504,582,510,5,f19421c1d4aa40978ebb69ca19b0e20d
2906b810c7d4411798c6938adc9daaa5,e2127556f4f64592b11af22de27a7932,0,18,NaN,NaN,2906b810c7d4411798c6938adc9daaa5
...,...,...,...,...,...,...
9b98b8c7a33c4b65b9aebfe6a799e6d9,9dc1421481194dcd9400aec7c9ae6366,408,516,414,5,9b98b8c7a33c4b65b9aebfe6a799e6d9
ae264e3637204a6fb9bb56bc8210ddfd,9dc1421481194dcd9400aec7c9ae6366,576,624,594,10,ae264e3637204a6fb9bb56bc8210ddfd
2298d6c36e964ae4a3e7e9706d1fb8c2,e4052622e5ba45a8b96b59aba68cf068,0,6,54,3,2298d6c36e964ae4a3e7e9706d1fb8c2
3f207df678b143eea3cee63160fa8bed,e4052622e5ba45a8b96b59aba68cf068,576,NaN,NaN,NaN,3f207df678b143eea3cee63160fa8bed


In [76]:
offers_df.to_csv("processed_data/offers_processed.csv")

In [108]:
offers_profile = pd.merge(offers_df, profile, on=['person'], how='left')

In [111]:
offers_profile_complete = pd.merge(offers_profile, portfolio, on=['offer'], how='left')
offers_profile_complete.drop(['channels'], inplace=True, axis=1)
offers_profile_complete

,person,offer received,offer viewed,offer completed,reward_x,offer,gender,age,became_member_on,income,reward_y,difficulty,duration,offer_type,email,mobile,social,web
0,78afa995795e4d85b5d9ceeca43f5fef,0,6,132,5,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,20170509,100000.0,5,5,7,bogo,1,1,0,1
1,78afa995795e4d85b5d9ceeca43f5fef,168,216,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837,F,75,20170509,100000.0,0,0,3,informational,1,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,408,408,510,10,ae264e3637204a6fb9bb56bc8210ddfd,F,75,20170509,100000.0,10,10,7,bogo,1,1,1,0
3,78afa995795e4d85b5d9ceeca43f5fef,504,582,510,5,f19421c1d4aa40978ebb69ca19b0e20d,F,75,20170509,100000.0,5,5,5,bogo,1,1,1,1
4,e2127556f4f64592b11af22de27a7932,0,18,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,M,68,20180426,70000.0,2,10,7,discount,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54862,9dc1421481194dcd9400aec7c9ae6366,408,516,414,5,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,83,20160307,50000.0,5,5,7,bogo,1,1,0,1
54863,9dc1421481194dcd9400aec7c9ae6366,576,624,594,10,ae264e3637204a6fb9bb56bc8210ddfd,F,83,20160307,50000.0,10,10,7,bogo,1,1,1,0
54864,e4052622e5ba45a8b96b59aba68cf068,0,6,54,3,2298d6c36e964ae4a3e7e9706d1fb8c2,F,62,20170722,82000.0,3,7,7,discount,1,1,1,1
54865,e4052622e5ba45a8b96b59aba68cf068,576,NaN,NaN,NaN,3f207df678b143eea3cee63160fa8bed,F,62,20170722,82000.0,0,0,4,informational,1,1,0,1


In [113]:
offers_profile_complete.rename(columns={'reward_x': 'reward_received', 'reward_y': 'reward_to_receive'}, inplace=True)

In [114]:
offers_df.to_csv("processed_data/processed_offers.csv")